In [232]:
import pandas as pd
import psycopg2
import os
from sqlalchemy import create_engine, text, MetaData, Table

### STEP 1: Load data

#### STEP 1A: Loading Existing Data from base tables (Project, Source, Types)

In [233]:
db_params = {
    'host': os.getenv('DB_HOST') or 'localhost',
    'database': os.getenv('DB_NAME') or 'tbmc_db',
    'user': os.getenv('DB_USER') or 'tbmc_db_user',
    'password': os.getenv('DB_PASSWORD') or '123456',
    'table': os.getenv('DB_TABLE') or 'tbmc_db1',
    'port': os.getenv('DB_PORT') or '5432'
}

In [234]:
def connect_to_database(db_params):
    try:
        conn = psycopg2.connect(
            host=db_params['host'],
            database=db_params['database'],
            user=db_params['user'],
            password=db_params['password']
        )
        conn.set_session(autocommit=True)
        
        engine = create_engine(f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")
        
        if conn and engine:
            print("Connection successful!")
            return conn, engine
        else:
            print("Connection failed.")

        
    except Exception as e:
        print(f"Error connecting to database: {e}")
        return None, None

conn, engine = connect_to_database(db_params)



Connection successful!


In [235]:
def existing_table_data():
    if engine:
        query_type = "SELECT * FROM type"
        query_source = "SELECT * FROM source"
        query_items = "SELECT * FROM items"

        item_type = pd.read_sql(query_type, engine)
        item_source = pd.read_sql(query_source, engine)
        item_items = pd.read_sql(query_items, engine)


        return item_type, item_source, item_items

    else:
        print("Failed to connect to the database.")

type_table, source_table, items_table = existing_table_data()

In [236]:
item_columns = list(items_table)
print(item_columns)

['description', 'man_r', 'unit', 'qty', 'unit_cost', 'amount', 'project', 'type', 'date', 'source']


#### STEP 1A: Loading New Data

In [237]:
data = pd.read_excel("../data/quotation_princing_analysis/DRY BLENDING PROJECT PHASE 1 STRUCTURE - EDP.xlsx", names=item_columns)

In [238]:
data

,description,man_r,unit,qty,unit_cost,amount,project,type,date,source
0,site enclosure,NaN,lot,1,1500,1500,DRY BLENDING PROJECT PHASE-1 STRUCTURE,Safety Provisions,2024-04-10,EDP
1,ppe requirements,NaN,lot,1,4500,4500,DRY BLENDING PROJECT PHASE-1 STRUCTURE,Safety Provisions,2024-04-10,EDP
2,cotton gloves,NaN,pcs,75,50,3750,DRY BLENDING PROJECT PHASE-1 STRUCTURE,Safety Provisions,2024-04-10,EDP
3,cuation tape,NaN,pcs,1,50,50,DRY BLENDING PROJECT PHASE-1 STRUCTURE,Safety Provisions,2024-04-10,EDP
4,rags,NaN,pcs,3,1500,4500,DRY BLENDING PROJECT PHASE-1 STRUCTURE,Safety Provisions,2024-04-10,EDP
5,welding gloves,NaN,pcs,1,250,250,DRY BLENDING PROJECT PHASE-1 STRUCTURE,Safety Provisions,2024-04-10,EDP
6,welding apron,NaN,pcs,1,450,450,DRY BLENDING PROJECT PHASE-1 STRUCTURE,Safety Provisions,2024-04-10,EDP
7,fire blanket,NaN,pcs,1,150,150,DRY BLENDING PROJECT PHASE-1 STRUCTURE,Safety Provisions,2024-04-10,EDP
8,goggles,NaN,pcs,5,150,750,DRY BLENDING PROJECT PHASE-1 STRUCTURE,Safety Provisions,2024-04-10,EDP
9,dust mask ( 50 pcs/box ),NaN,box,75,80,6000,DRY BLENDING PROJECT PHASE-1 STRUCTURE,Safety Provisions,2024-04-10,EDP


In [239]:
if engine:    
    # Append the DataFrame to the table named 'items' in the connected database
    data[columns_to_insert].to_sql('items', engine, if_exists='append', index=False)
    print("DataFrame appended to the database successfully!")
else:
    print("Failed to connect to the database.")


DataFrame appended to the database successfully!


### STEP 2: Final cleaning

### STEP 3: Add project name to "Project" table

#### STEP 3A: Connect to the database

#### STEP 3B: Upload data

#### STEP 3C: Close Connection

In [240]:
conn.close()

In [241]:
if conn:
    conn.close()
    print("Connection closed.")

if engine:
    engine.dispose()
    print("Engine disposed.")

Connection closed.
Engine disposed.
